In [3]:
!pip install transformers

In [4]:

import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import shutil
import sys   

In [5]:
train_path = "/kaggle/input/code-injection-6labels-balance/code_injection_6labels_balance.csv"

In [6]:
df = pd.read_csv(train_path)

In [7]:
target_list = ["000 - Normal", '126 - Path Traversal', 
               '242 - Code Injection', '274 - HTTP Verb Tampering', 
               '66 - SQL Injection', '88 - OS Command Injection']

In [8]:
train, validate, test = np.split(df.sample(frac=1, random_state=42),[int(.6*len(df)), int(.8*len(df))])

/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [9]:
# hyperparameters
MAX_LEN = 256
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 2
LEARNING_RATE = 1e-05

In [10]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModel
from transformers import BertTokenizer, BertModel
from transformers import DistilBertTokenizer, DistilBertModel

In [11]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [12]:
class CustomDataset(torch.utils.data.Dataset):

    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.title = df['text']
        self.targets = self.df[target_list].values
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.targets[index])
        }

In [13]:
train = train.reset_index(drop=True)
validate = validate.reset_index(drop=True)
test = test.reset_index(drop=True)

train = CustomDataset(train, tokenizer, MAX_LEN)
validate= CustomDataset(validate, tokenizer, MAX_LEN)
test = CustomDataset(test, tokenizer, MAX_LEN)

In [14]:
train_data_loader = torch.utils.data.DataLoader(train, 
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
    num_workers=0
)

val_data_loader = torch.utils.data.DataLoader(validate, 
    batch_size=VALID_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)
test_data_loader = torch.utils.data.DataLoader(test, 
    batch_size=VALID_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

In [15]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [16]:
def load_ckp(checkpoint_fpath, model):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into       
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
#     optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
#     valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss 
    return model
#     , optimizer, checkpoint['epoch'], valid_loss_min.item()

def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

In [26]:
model = load_ckp('/kaggle/input/bert-for-capec-dataset/pytorch/bertbase/1/BERT_CAPEC.pt', model)

In [17]:
from transformers import RobertaModel, RobertaTokenizer

In [18]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bert_model = AutoModel.from_pretrained("bert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, 6)
    
    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.bert_model(
            input_ids, 
            attention_mask=attn_mask, 
            token_type_ids=token_type_ids
        )
        hidden_state = output[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.linear(pooler)
        return output

model = BERTClass()

model.to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [20]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [21]:
val_targets=[]
val_outputs=[]

In [22]:
def train_model(n_epochs, training_loader, validation_loader, model, 
                optimizer, checkpoint_path, best_model_path):
   
  # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf
   
 
    for epoch in range(1, n_epochs+1):
        train_loss = 0
        valid_loss = 0
        model.train()
        print('############# Epoch {}: Training Start   #############'.format(epoch))
        for batch_idx, data in enumerate(training_loader):
            #print('yyy epoch', batch_idx)
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)

            outputs = model(ids, mask, token_type_ids)

            optimizer.zero_grad()
            loss = loss_fn(outputs, targets)
            #if batch_idx%5000==0:
             #   print(f'Epoch: {epoch}, Training Loss:  {loss.item()}')

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            #print('before loss data in training', loss.item(), train_loss)
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))
            #print('after loss data in training', loss.item(), train_loss)
            
    print('############# Epoch {}: Training End     #############'.format(epoch))
    
    print('############# Epoch {}: Validation Start   #############'.format(epoch))
    ######################    
    # validate the model #
    ######################
 
    model.eval()
   
    with torch.no_grad():
        for batch_idx, data in enumerate(validation_loader, 0):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)

            loss = loss_fn(outputs, targets)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))
            val_targets.extend(targets.cpu().detach().numpy().tolist())
            val_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

        print('############# Epoch {}: Validation End     #############'.format(epoch))
        # calculate average losses
        #print('before cal avg train loss', train_loss)
        if len(training_loader) > 0:
            train_loss = train_loss/len(training_loader)
        if len(validation_loader) > 0:
            valid_loss = valid_loss/len(validation_loader)
        # print training/validation statistics 
        print('Epoch: {} \tAvgerage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))

        # create checkpoint variable and add important data
        checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': valid_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
        }

        # save checkpoint
        save_ckp(checkpoint, False, checkpoint_path, best_model_path)

        ## TODO: save the model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
            # save checkpoint as best model
            save_ckp(checkpoint, True, checkpoint_path, best_model_path)
            valid_loss_min = valid_loss

        print('############# Epoch {}  Done   #############\n'.format(epoch))

        return model

In [23]:
ckpt_path = "/kaggle/working/curr_ckpt"
best_model_path = "/kaggle/working/best_model.pt"

In [24]:
import time
t = time.time()
trained_model = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer, ckpt_path, best_model_path)
print(time.time()-t)

############# Epoch 1: Training Start   #############
############# Epoch 2: Training Start   #############
############# Epoch 2: Training End     #############
############# Epoch 2: Validation Start   #############
############# Epoch 2: Validation End     #############
Epoch: 2 	Avgerage Training Loss: 0.000012 	Average Validation Loss: 0.000028
Validation loss decreased (inf --> 0.000028).  Saving model ...
############# Epoch 2  Done   #############

3545.7485885620117


In [20]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report,accuracy_score, f1_score

In [21]:
def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
#     y_true1 = y_true.to_numpy()
#     y_pred1 = y_pred.toarray()
    acc_list = []
    
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/float(len(set_true.union(set_pred)))
        acc_list.append(tmp_a)
    return np.mean(acc_list)

In [22]:
# from sklearn.metrics import hamming_loss,classification_report
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report,accuracy_score, f1_score, hamming_loss

In [23]:
def save_classification(y_test, y_pred, labels):
    if isinstance(y_pred, np.ndarray) == False:
        y_pred = y_pred.toarray()

    def accuracy(y_true, y_pred):
        temp = 0
        for i in range(y_true.shape[0]):
            numerator = sum(np.logical_and(y_true[i], y_pred[i]))
            denominator = sum(np.logical_or(y_true[i], y_pred[i]))
            if denominator != 0:
                temp += numerator / denominator
        return temp / y_true.shape[0]

    out = classification_report(y_test,y_pred, output_dict=True, target_names=labels)
    total_support = out['samples avg']['support']

    mr = accuracy_score(y_test, y_pred)
    acc = accuracy(y_test,y_pred)
    hm = hamming_loss(y_test, y_pred)

    out['Exact Match Ratio'] = {'precision': mr, 'recall': mr, 'f1-score': mr, 'support': total_support}
    out['Hamming Loss'] = {'precision': hm, 'recall': hm, 'f1-score': hm, 'support': total_support}
    out['Accuracy'] = {'precision': acc, 'recall': acc, 'f1-score': acc, 'support': total_support}
    out_df = pd.DataFrame(out).transpose()
    print(out_df)

    

    return out_df

In [24]:
def predict(testing_loader, model):
    print("\nPredicting...")
    # deactivate dropout layers
    model.eval()

    # empty list to save the model predictions
    total_preds = []
    total_labels = []
    # iterate over batches
    for step, batch in enumerate(testing_loader):
        # push the batch to gpu
        ids = batch['input_ids'].to(device)
        mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        targets = batch['targets'].to(device)

        # deactivate autograd
        with torch.no_grad():
            # model predictions
            output = model(ids, mask, token_type_ids)
            final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
            
            total_preds += list(final_output)
            total_labels += targets.tolist()

    return total_labels, total_preds

In [27]:
import time

In [29]:
t = time.time()
total_labels, total_preds = predict(test_data_loader, model)
print(time.time()-t)



Predicting...
206.6841902732849


In [28]:
save_classification(y_test=np.array(np.nan_to_num(total_labels)), y_pred=np.array(np.round(total_preds)), labels=target_list)

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                           precision    recall  f1-score  support
000 - Normal                0.984889  0.978645  0.981757   2997.0
126 - Path Traversal        0.999676  0.970135  0.984684   3181.0
242 - Code Injection        1.000000  0.997442  0.998719   3127.0
274 - HTTP Verb Tampering   1.000000  1.000000  1.000000   1111.0
66 - SQL Injection          0.989886  0.969639  0.979658   3129.0
88 - OS Command Injection   1.000000  0.981545  0.990686   1463.0
micro avg                   0.994796  0.980744  0.987720  15008.0
macro avg                   0.995742  0.982901  0.989251  15008.0
weighted avg                0.994805  0.980744  0.987695  15008.0
samples avg                 0.979996  0.979852  0.979900  15008.0
Exact Match Ratio           0.979709  0.979709  0.979709  15008.0
Hamming Loss                0.004374  0.004374  0.004374  15008.0
Accuracy                    0.979852  0.979852  0.979852  15008.0


,precision,recall,f1-score,support
000 - Normal,0.984889,0.978645,0.981757,2997.0
126 - Path Traversal,0.999676,0.970135,0.984684,3181.0
242 - Code Injection,1.000000,0.997442,0.998719,3127.0
274 - HTTP Verb Tampering,1.000000,1.000000,1.000000,1111.0
66 - SQL Injection,0.989886,0.969639,0.979658,3129.0
88 - OS Command Injection,1.000000,0.981545,0.990686,1463.0
micro avg,0.994796,0.980744,0.987720,15008.0
macro avg,0.995742,0.982901,0.989251,15008.0
weighted avg,0.994805,0.980744,0.987695,15008.0
samples avg,0.979996,0.979852,0.979900,15008.0
